# Predict TripAdvisor Rating

In this competition we're provided with:
- train and test datasets of 40,000 and 10,000 samples respectivly and 9 features except target feature;
- target feature is **Rating** - discrete number.
- model with hyperparameters that we're **not allowed** to change (**RandomForestRegressor**: {'bootstrap': **True**, 'criterion': '**mse**', 'max_features': '**auto**', 'n_estimators': 100});
- baseline that gives **MAE** = 0.212

**What has been done**:
1. New features:
    - restaurant type, area food, country food, spcialities were get from cuisine style feature
    - name of restaurant and its region were get from url
    - chain and chain_scale were get by grouping restaurants' names
    - lasts 2 reviews date delta and last review date delta with max date of review as well as positive and negative words scores were get from reviews feature
    - some normalized and scaled be city numerical features
    - some means and medians
    - polinomial features
    - dummy features
2. 27 tests of model with different features conbinations
3. **MAE** has been reduced to **0.198** and after rounding predict with pitch 0.5 to make continious number discrete - to **0.168** on validation data (**0.1726** on the submission)

- For data investigation results go to 'Bella K [SF TripAdvisor Rating] EDA+NaNs'
- For models testing results go to 'Bella K [SF TripAdvisor Rating] FE+Models'

# import

In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/sf-dst-restaurant-rating/sample_submission.csv
/kaggle/input/sf-dst-restaurant-rating/main_task.csv
/kaggle/input/sf-dst-restaurant-rating/kaggle_task.csv
/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/model_data.csv
/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/best_model.pkl
/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/data_for_model.csv
/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/__results__.html
/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/__notebook_source__.ipynb
/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/requirements.txt
/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/__notebook__.ipynb
/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/lists_dict.json
/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/__output__.json
/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/custom.css
/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/__results___files/__results___23_4.png
/kaggle/input/be

In [2]:
from matplotlib import rcParams
#%config InlineBackend.figure_format = 'svg' # graphs in svg look clearer
plt.style.use('ggplot')  # beautiful graphs
plt.rcParams['figure.figsize'] = (12, 4)  # figure size

import warnings
warnings.filterwarnings("ignore")

from datetime import datetime
from datetime import date

import json

In [3]:
# make our experiments reproducible
RANDOM_SEED = 42
CURRENT_DATE = pd.to_datetime(date.today().strftime('%Y-%m-%d'))
CURRENT_DATE

Timestamp('2023-05-22 00:00:00')

In [4]:
# fix the version of the packages
!pip freeze > requirements.txt

# DATA

In [13]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
#df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

print('Size of the test dataset:', df_test.shape)
print('Size of the submission dataset:', sample_submission.shape, end='\n\n')

model_data = pd.read_csv('/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/model_data.csv')
print('Size of the model dataset:', model_data.shape)

with open('/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/lists_dict.json', 'rb') as infile:
    lists_dict = json.load(infile)
    
len(lists_dict['rest_type_list'])

Size of the test dataset: (10000, 9)
Size of the submission dataset: (10000, 2)

Size of the model dataset: (40000, 74)


19

In [14]:
df_test.head()

,Restaurant_id,City,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_0,Paris,"['Bar', 'Pub']",12963.0,$$ - $$$,4.0,"[[], []]",/Restaurant_Review-g187147-d10746918-Reviews-L...,d10746918
1,id_1,Helsinki,"['European', 'Scandinavian', 'Gluten Free Opti...",106.0,$$ - $$$,97.0,"[['Very good reviews!', 'Fine dining in Hakani...",/Restaurant_Review-g189934-d6674944-Reviews-Ra...,d6674944
2,id_2,Edinburgh,['Vegetarian Friendly'],810.0,$$ - $$$,28.0,"[['Better than the Links', 'Ivy Black'], ['12/...",/Restaurant_Review-g186525-d13129638-Reviews-B...,d13129638
3,id_3,London,"['Italian', 'Mediterranean', 'European', 'Vege...",1669.0,$$$$,202.0,"[['Most exquisite', 'Delicious and authentic']...",/Restaurant_Review-g186338-d680417-Reviews-Qui...,d680417
4,id_4,Bratislava,"['Italian', 'Mediterranean', 'European', 'Seaf...",37.0,$$$$,162.0,"[['Always the best in bratislava', 'Very good ...",/Restaurant_Review-g274924-d1112354-Reviews-Ma...,d1112354


In [15]:
sample_submission.head()

,Restaurant_id,Rating
0,id_0,2.0
1,id_1,2.5
2,id_2,4.0
3,id_3,1.0
4,id_4,4.0


In [16]:
model_data.head()

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta,...,city_cheap,city_medium,city_expensive,city_chain_scaled,city_cheap_scaled,city_medium_scaled,city_expensive_scaled,rev_num_city_scaled_old,reviews_pos_score,reviews_neg_score
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.5,194.0,"[""'Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,...,594,4065,238,0.114968,0.121299,0.830100,0.048601,0.04,1.0,0.0
1,id_1535,Stockholm,NaN,1537.0,4.0,2.5,10.0,"[""'Unique cuisine', 'Delicious Nepalese food'""...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,...,52,743,25,0.080488,0.063415,0.906098,0.030488,0.01,0.0,0.0
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,4.0,688.0,"[""'Catch up with friends', 'Not exceptional'"",...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,...,1064,4479,214,0.159458,0.184818,0.778009,0.037172,0.12,0.0,0.0
3,id_3456,Berlin,NaN,3458.0,5.0,2.5,3.0,NaN,/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,...,340,1766,49,0.090023,0.157773,0.819490,0.022738,0.00,NaN,NaN
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2.5,84.0,"[""'Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,...,91,761,41,0.124300,0.101904,0.852184,0.045913,0.09,0.0,0.0


# Preprocessing

In [9]:
# FUNCTIONS

import collections

# count the number of mentions of unique values and get those that come across more than 1 time
def collect_repeated_items(col):
    c = collections.Counter()
    for el in data[col].tolist():
        c[el]+=1
    print('Unique values:', len(c))
    
    new_c = {k: v for k, v in c.items() if v > 1}
    print('Values repeated more then one time:', len(new_c))
    
    c_list = list(new_c.keys())
    
    return c_list



import math

# divide all restaurants into conditional groups by frequency of occurrence
def get_param_scale(cell):
    for el in percentile_75_list:
        if cell < el+1:
            cell = el
            break
    
    return cell


# take the percentile as a criterion
def get_percentile(pc, df, col):
    percentile_list = []
    percentile = 0
    col_max = max(df[col])

    while math.ceil(col_max) > math.ceil(percentile):
        percentile = math.ceil(np.percentile(df.loc[(df[col]>percentile)][col], pc))
        percentile_list.append(percentile)
        
    return percentile_list



# in the case of chain==1, rest_type, area_food, country_food, specialties will be filled with mods by city and name
def get_mode(col):

    city_name_dict_1 = {}
    for city in model_data.city.unique():
        city_name_dict_1.update({city: {'name': model_data.loc[(model_data['city']==city)&(model_data['chain']==1)&(model_data['name'].isna()==False)].groupby('name')[col].apply(lambda x: x.mode()).reset_index().to_dict('index')}})

    city_name_dict_1 = {k: v['name'] for k, v in city_name_dict_1.items()}
    city_name_dict_1 = {key: {v['name']: v[col] for k, v in city_name_dict_1[key].items()} for key, value in city_name_dict_1.items()}

    city_name_dict_0 = {}
    for city in df_test.city.unique():
        city_name_dict_0.update({city: {'name': df_test.loc[(df_test['city']==city)&(df_test['chain']==1)&(df_test['name'].isna()==False)].groupby('name')[col].apply(lambda x: x.mode()).reset_index().to_dict('index')}})

    city_name_dict_0 = {k: v['name'] for k, v in city_name_dict_0.items()}
    city_name_dict_0 = {key: {v['name']: v[col] for k, v in city_name_dict_0[key].items()} for key, value in city_name_dict_0.items()}

    return city_name_dict_1, city_name_dict_0
    # {'Paris': {'231_east_street': 'Fast Food', ...}}
    
    
# in case of chain==0 rest_type_count, area_food_court, country_food_count, specialities_count
# fill in median by city and price_range
def get_median(col):

    city_price_dict_1 = {}
    for city in model_data.city.unique():
        city_price_dict_1.update({city: {'price_range': model_data.loc[(model_data['city']==city)&(model_data['chain']==0)&(model_data['name'].isna()==False)].groupby('price_range')[col].median().reset_index().to_dict('index')}})

    city_price_dict_1 = {k: v['price_range'] for k, v in city_price_dict_1.items()}
    city_price_dict_1 = {key: {v['price_range']: round(v[col], 0) for k, v in city_price_dict_1[key].items()} for key, value in city_price_dict_1.items()}

    city_price_dict_0 = {}
    for city in df_test.city.unique():
        city_price_dict_0.update({city: {'price_range': df_test.loc[(df_test['city']==city)&(df_test['chain']==0)&(df_test['name'].isna()==False)].groupby('price_range')[col].median().reset_index().to_dict('index')}})

    city_price_dict_0 = {k: v['price_range'] for k, v in city_price_dict_0.items()}
    city_price_dict_0 = {key: {v['price_range']: round(v[col], 0) for k, v in city_price_dict_0[key].items()} for key, value in city_name_dict_0.items()}

    return city_price_dict_1, city_price_dict_0
    # 'Krakow': {1.0: 1.0, 2.5: 1.0, 4.0: 2.0}
    
    
    
from itertools import product

def get_stat_dict(df, cat, num, stat): # stat_list = ['mean', 'median', 'std', 'quant_25', 'quant_75']
    cat_num_dict = {}
    
    if stat=='mean':
        for city in df.city.unique():
            try:
                cat_num_dict.update({city: {str(cat): df.loc[df['city']==city].groupby(cat)[num].mean().reset_index().to_dict('index')}})
            except: pass
        
    elif stat=='median':
        for city in df.city.unique():
            try:
                cat_num_dict.update({city: {str(cat): df.loc[df['city']==city].groupby(cat)[num].median().reset_index().to_dict('index')}})
            except: pass
        
    cat_num_dict = {k: v[str(cat)] for k, v in cat_num_dict.items()}
    cat_num_dict = {key: {v[str(cat)]: round(v[str(num)], 2) for k, v in cat_num_dict[key].items()} for key, value in cat_num_dict.items()}
    
    return cat_num_dict



# make a dictionary of the Cuisine Style frequency taking into account a certain parameter
def cuisine_count_dict(cell):
    c = collections.Counter()
    for el in cell:
        if el is not np.nan:
            if type(el)==list:
                for x in el:
                    c[x]+=1
            else:
                c[el]+=1
    
    c = {k: v for k, v in sorted(c.items(), key=by_value, reverse=True)}
    return c


# sort the dictionary in descending order of quantity
def by_value(item):
    return item[1]


def find_item(cell):
    if item in cell:
        return 1
    return 0



# count the number of positive and negative words in the reviews
import re
import nltk
from nltk.corpus import stopwords

def get_words_list(cell):
    cell_text = ' '.join(cell)
    cell_text_new = re.sub(r'[.,?!:;()=-_\'\"]', '', cell_text)
    cell_words_list = cell_text_new.split(' ')
    cell_words_list = [w.strip() for w in cell_words_list if len(w) > 1 ]
    cell_words_list = [w.strip() for w in cell_words_list if w not in stopwords.words('english')]
    
    return cell_words_list


def get_positive_score(cell):
    cell_score = sum([1 for w in cell if w in lists_dict['pos_list']])
    
    return cell_score


def get_negative_score(cell):
    cell_score = sum([1 for w in cell if w in lists_dict['neg_list']])
    
    return cell_score



def get_data_columns(list_of_lists):
    columns_list = []
    for el in list_of_lists:
        columns_list.extend(el)
        
    return columns_list


def remove_elments(old_list, el_list):
    for el in el_list:
        old_list.remove(el)
        
    return old_list


def get_data_columns_dict(list_of_names):
    columns_list = []
    for name in list_of_names:
        for el in features_dict[name]:
            columns_list.extend([el])
            
    return columns_list


def features_dict_update(feature_list_name, new_feature):
    feature_list = features_dict[feature_list_name]
    feature_list.extend([new_feature])
    features_dict.update({feature_list_name: feature_list})

    


In [17]:
# IMPORTANT!!! Everything is calculated according to train and transferred to test
# and only if there is no such combination in train, then it is calculated according to test df_test

#def preproc_test_data(df_input, model_data):
    
    # DATA
    #df_output = df_input.copy()
    
print('Data processing started')
    
df_test['Rating'] = 0 # in test we don't have column Rating, it's our target
df_test.columns = [x.lower().replace(' ', '_') for x in df_test.columns]

# the structure of the list, but the data type is a string
# turn it into a list
df_test['cuisine_style'] = df_test['cuisine_style'].dropna().apply(lambda x: x[2:-2].split("', '"))
# count the number of kitchens
df_test['cuisine_style_count'] = df_test['cuisine_style'].dropna().apply(lambda x: len(x))

df_test['price_range'] = df_test['price_range'].dropna().apply(lambda x: 1.0 if x=='$' else (2.5 if x=='$$ - $$$' else(4.0 if x=='$$$$' else x)))

    
city_count_1 = model_data['city'].value_counts()
city_count_0 = df_test['city'].value_counts()
df_test['city_count'] = df_test['city'].dropna().apply(lambda x: city_count_1[x] if x in city_count_1.index else (city_count_0[x] if x in city_count_0.index else np.nan))

    
# get the data about the name of the restaurant (or chain?), the region and some other g-code from URL_TA
df_test['name'] = df_test['url_ta'].dropna().apply(lambda x: x.split('-')[4].replace('.html', ''))
df_test['region'] = df_test['url_ta'].dropna().apply(lambda x: x.split('-')[5].replace('.html', '') if len(x.split('-'))>5 else x.split('-')[4].replace('.html', ''))
df_test['g_code'] = df_test['url_ta'].dropna().apply(lambda x: x.split('-')[1])
# in some cases, there is no indication of the name, then the region is pulled up - let's fix it
df_test.iloc[df_test.loc[df_test['name'] == df_test['region']].index.tolist(), df_test.columns.tolist().index('name')] = np.nan
df_test['name'] = df_test['name'].dropna().apply(lambda x: x.lower())
df_test['region'] = df_test['region'].dropna().apply(lambda x: x.lower())

    
name_count_1 = model_data.loc[(model_data['name'].isna()==False)]['name'].value_counts()
name_count_0 = df_test.loc[(df_test['name'].isna()==False)]['name'].value_counts()
df_test['name_count'] = df_test['name'].apply(lambda x: name_count_1[x] if x in name_count_1.index else (name_count_0[x] if x in name_count_0.index else 1))
    
model_data['name_count'] = model_data['name'].apply(lambda x: name_count_1[x] if x in name_count_1.index else 1)

    
df_test['chain'] = df_test['name_count'].dropna().apply(lambda x: 0 if x==1 else 1)
# take 75 percentile as a criterion
percentile_75_list = get_percentile(75, model_data, 'name_count') # chain_scale
df_test['chain_scale'] = df_test['name_count'].dropna().apply(get_param_scale)

    
gcode_count_1 = model_data['g_code'].value_counts()
gcode_count_0 = df_test['g_code'].value_counts()
df_test['gcode_count'] = df_test['g_code'].dropna().apply(lambda x: gcode_count_1[x] if x in gcode_count_1.index else (gcode_count_0[x] if x in gcode_count_0.index else np.nan))


# the structure of the lists in the list (review and dates), but the data type is a string
# turning empty lists into nan and strings into lists
df_test['reviews'] = df_test['reviews'].dropna().apply(lambda x: np.nan if x == '[[], []]' else x[2:-2].split("], ["))

# at the same time, we separate the review and turn the data into lists
df_test['reviews_text'] = df_test['reviews'].dropna().apply(lambda x: x[0][1:-1].split("', '"))

# separate the dates from the review and turn the data into lists
df_test['reviews_date'] = df_test['reviews'].dropna().apply(lambda x: x[1][1:-1].split("', '"))
# turning strings into dates
df_test['reviews_date'] = df_test['reviews_date'].dropna().apply(lambda x: [pd.to_datetime(i) for i in x])
df_test['reviews_date_last'] = df_test['reviews_date'].dropna().apply(lambda x: max(x))
    
model_data['reviews_date_last'] = model_data['reviews_date_last'].dropna().apply(lambda x: pd.to_datetime(x))
    
    
reviews_date_max = max([max(model_data['reviews_date_last'].dropna().tolist()), max(df_test['reviews_date_last'].dropna().tolist())])
df_test['reviews_date_delta'] = df_test['reviews_date'].dropna().apply(lambda x: (max(x) - min(x)).days)
df_test['reviews_date_delta_max'] = df_test['reviews_date'].dropna().apply(lambda x: (reviews_date_max - max(x)).days)

    
# divide cuisine_style
df_test['rest_type'] = df_test['cuisine_style'].dropna().apply(lambda x: [st for st in lists_dict['rest_type_list'] if st in x])
df_test['rest_type_count'] = df_test['rest_type'].dropna().apply(lambda x: len(x))
df_test['rest_type'] = df_test['rest_type'].dropna().apply(lambda x: ', '.join(x) if len(x)>0 else np.nan)

df_test['area_food'] = df_test['cuisine_style'].dropna().apply(lambda x: [st for st in lists_dict['area_food_list'] if st in x])
df_test['area_food_count'] = df_test['area_food'].dropna().apply(lambda x: len(x))
df_test['area_food'] = df_test['area_food'].dropna().apply(lambda x: ', '.join(x) if len(x)>0 else np.nan)

df_test['country_food'] = df_test['cuisine_style'].dropna().apply(lambda x: [st for st in lists_dict['country_food_list'] if st in x])
df_test['country_food_count'] = df_test['country_food'].dropna().apply(lambda x: len(x))
df_test['country_food'] = df_test['country_food'].dropna().apply(lambda x: ', '.join(x) if len(x)>0 else np.nan)

df_test['spcialities'] = df_test['cuisine_style'].dropna().apply(lambda x: [st for st in lists_dict['spcialities_list'] if st in x])
df_test['spcialities_count'] = df_test['spcialities'].dropna().apply(lambda x: len(x))
df_test['spcialities'] = df_test['spcialities'].dropna().apply(lambda x: ', '.join(x) if len(x)>0 else np.nan)


Data processing started


In [18]:
# NaNs
    
print('NaNs processing started')

# price_range

def round_to_closest(number, lst=[1.0, 2.5, 4.0]):
    return min(lst, key=lambda x: abs(x - number))

price_range_dict = {}
for city in model_data.city.unique():
    price_range_dict.update({city: {'chain_scale': model_data.loc[(model_data['city']==city)].groupby('chain_scale')['price_range'].mean().reset_index().to_dict('index')}})
        
price_range_dict = {k: v['chain_scale'] for k, v in price_range_dict.items()}
price_range_dict = {key: {v['chain_scale']: round_to_closest(v['price_range']) if math.isnan(v['price_range'])==False else np.nan for k, v in price_range_dict[key].items()} for key, value in price_range_dict.items()}

df_test['price_range_old'] = df_test['price_range']
df_test['price_range'] = df_test.apply(lambda x: price_range_dict[x['city']][x['chain_scale']] if math.isnan(x['price_range_old'])==True else x['price_range'], axis=1)
df_test['price_range'] = df_test['price_range'].fillna(2.5)

    
# number_of_reviews

num_rev_cuisine_dict = {}
for city in model_data.city.unique():
    num_rev_cuisine_dict.update({city: {'cuisine_style_count': model_data.loc[(model_data['city']==city)&(model_data['cuisine_style_count'].isna()==False)].groupby('cuisine_style_count')['number_of_reviews'].median().reset_index().to_dict('index')}})
        
num_rev_cuisine_dict = {k: v['cuisine_style_count'] for k, v in num_rev_cuisine_dict.items()}
num_rev_cuisine_dict = {key: {v['cuisine_style_count']: v['number_of_reviews'] if math.isnan(v['number_of_reviews'])==False else np.nan for k, v in num_rev_cuisine_dict[key].items()} for key, value in num_rev_cuisine_dict.items()}


num_rev_price_dict = {}
for city in model_data.city.unique():
    num_rev_price_dict.update({city: {'price_range': model_data.loc[(model_data['city']==city)].groupby('price_range')['number_of_reviews'].median().reset_index().to_dict('index')}})
        
num_rev_price_dict = {k: v['price_range'] for k, v in num_rev_price_dict.items()}
num_rev_price_dict = {key: {v['price_range']: v['number_of_reviews'] if math.isnan(v['number_of_reviews'])==False else np.nan for k, v in num_rev_price_dict[key].items()} for key, value in num_rev_price_dict.items()}


df_test['number_of_reviews_old'] = df_test['number_of_reviews']

df_test['number_of_reviews'] = df_test.apply(lambda x: num_rev_cuisine_dict[x['city']][x['cuisine_style_count']] if math.isnan(x['number_of_reviews_old'])==True and math.isnan(x['cuisine_style_count'])==False else x['number_of_reviews'], axis=1)
df_test['number_of_reviews'] = df_test.apply(lambda x: num_rev_price_dict[x['city']][x['price_range']] if math.isnan(x['number_of_reviews'])==True else x['number_of_reviews'], axis=1)

    
# cuisine_style

# instead of name_count, let's count name_city_count
city_name_dict_1 = {}
for city in model_data.city.unique():
    city_name_dict_1.update({city: {'name': model_data.loc[(model_data['city']==city)&(model_data['name'].isna()==False)].groupby('name')['id_ta'].count().reset_index().to_dict('index')}})

city_name_dict_1 = {k: v['name'] for k, v in city_name_dict_1.items()}
city_name_dict_1 = {key: {v['name']: v['id_ta'] if math.isnan(v['id_ta'])==False else np.nan for k, v in city_name_dict_1[key].items()} for key, value in city_name_dict_1.items()}

city_name_dict_0 = {}
for city in df_test.city.unique():
    city_name_dict_0.update({city: {'name': df_test.loc[(df_test['city']==city)&(df_test['name'].isna()==False)].groupby('name')['id_ta'].count().reset_index().to_dict('index')}})

city_name_dict_0 = {k: v['name'] for k, v in city_name_dict_0.items()}
city_name_dict_0 = {key: {v['name']: v['id_ta'] if math.isnan(v['id_ta'])==False else np.nan for k, v in city_name_dict_0[key].items()} for key, value in city_name_dict_0.items()}

# {'Paris': {015_gang_nam': 1}, ...}

df_test['name_city_count'] = df_test.apply(lambda x: city_name_dict_1[x['city']][x['name']] if x['name'] in list(city_name_dict_1[x['city']].keys()) else (city_name_dict_0[x['city']][x['name']] if x['name'] in list(city_name_dict_0[x['city']].keys()) else 1), axis=1)
df_test = df_test.drop('name_count', axis=1)
#model_data = model_data.drop('name_count', axis=1)

    
df_test['rest_type_old'] = df_test['rest_type']
df_test['area_food_old'] = df_test['area_food']
df_test['country_food_old'] = df_test['country_food']
df_test['spcialities_old'] = df_test['spcialities']
    

# in the case of chain==1 rest_type, area_food, country_food, specialties fill in with modes by city and name
city_name_dict_1, city_name_dict_0 = get_mode('rest_type')
df_test['rest_type'] = df_test.apply(lambda x: city_name_dict_1[x['city']][x['name']] if x['name'] in list(city_name_dict_1[x['city']].keys()) and math.isnan(x['cuisine_style_count'])==True else (city_name_dict_0[x['city']][x['name']] if x['name'] in list(city_name_dict_0[x['city']].keys()) and math.isnan(x['cuisine_style_count'])==True else x['rest_type']), axis=1)

city_name_dict_1, city_name_dict_0 = get_mode('area_food')
df_test['area_food'] = df_test.apply(lambda x: city_name_dict_1[x['city']][x['name']] if x['name'] in list(city_name_dict_1[x['city']].keys()) and math.isnan(x['cuisine_style_count'])==True else (city_name_dict_0[x['city']][x['name']] if x['name'] in list(city_name_dict_0[x['city']].keys()) and math.isnan(x['cuisine_style_count'])==True else x['area_food']), axis=1)

city_name_dict_1, city_name_dict_0 = get_mode('country_food')
df_test['country_food'] = df_test.apply(lambda x: city_name_dict_1[x['city']][x['name']] if x['name'] in list(city_name_dict_1[x['city']].keys()) and math.isnan(x['cuisine_style_count'])==True else (city_name_dict_0[x['city']][x['name']] if x['name'] in list(city_name_dict_0[x['city']].keys()) and math.isnan(x['cuisine_style_count'])==True else x['country_food']), axis=1)

city_name_dict_1, city_name_dict_0 = get_mode('spcialities')
df_test['spcialities'] = df_test.apply(lambda x: city_name_dict_1[x['city']][x['name']] if x['name'] in list(city_name_dict_1[x['city']].keys()) and math.isnan(x['cuisine_style_count'])==True else (city_name_dict_0[x['city']][x['name']] if x['name'] in list(city_name_dict_0[x['city']].keys()) and math.isnan(x['cuisine_style_count'])==True else x['spcialities']), axis=1)

# recalculate all ..._count
df_test['rest_type_count'] = df_test['rest_type'].dropna().apply(lambda x: len(x.split(', ')))
df_test['area_food_count'] = df_test['area_food'].dropna().apply(lambda x: len(x.split(', ')))
df_test['country_food_count'] = df_test['country_food'].dropna().apply(lambda x: len(x.split(', ')))
df_test['spcialities_count'] = df_test['spcialities'].dropna().apply(lambda x: len(x.split(', ')))

    
# in the case of chain==0 cuisine_style_count, rest_type_count, area_food_count, country_food_count, spcialities_count, 
# fill in with median by city and price_range
city_price_dict_1, city_price_dict_0 = get_median('rest_type_count')
df_test['rest_type_count'] = df_test.apply(lambda x: city_price_dict_1[x['city']][x['price_range']] if math.isnan(x['cuisine_style_count'])==True else (city_price_dict_0[x['city']][x['price_range']] if math.isnan(x['cuisine_style_count'])==True else x['rest_type_count']), axis=1)

city_price_dict_1, city_price_dict_0 = get_median('area_food_count')
df_test['area_food_count'] = df_test.apply(lambda x: city_price_dict_1[x['city']][x['price_range']] if math.isnan(x['cuisine_style_count'])==True else (city_price_dict_0[x['city']][x['price_range']] if math.isnan(x['cuisine_style_count'])==True else x['area_food_count']), axis=1)

city_price_dict_1, city_price_dict_0 = get_median('country_food_count')
df_test['country_food_count'] = df_test.apply(lambda x: city_price_dict_1[x['city']][x['price_range']] if math.isnan(x['cuisine_style_count'])==True else (city_price_dict_0[x['city']][x['price_range']] if math.isnan(x['cuisine_style_count'])==True else x['country_food_count']), axis=1)

city_price_dict_1, city_price_dict_0 = get_median('spcialities_count')
df_test['spcialities_count'] = df_test.apply(lambda x: city_price_dict_1[x['city']][x['price_range']] if math.isnan(x['cuisine_style_count'])==True else (city_price_dict_0[x['city']][x['price_range']] if math.isnan(x['cuisine_style_count'])==True else x['spcialities_count']), axis=1)

# fill remain NaNs with 0 to recalculate cuisine_style_count
df_test['rest_type_count'] = df_test['rest_type_count'].fillna(0)
df_test['area_food_count'] = df_test['area_food_count'].fillna(0)
df_test['country_food_count'] = df_test['country_food_count'].fillna(0)
df_test['spcialities_count'] = df_test['spcialities_count'].fillna(0)

df_test['cuisine_style_count'] = df_test.apply(lambda x: x['rest_type_count']+x['area_food_count']+x['country_food_count']+x['spcialities_count'], axis=1)

    
# reviews_date_delta & reviews_date_delta_max
# We will fill in the gaps not in dates, but in intervals
rev_delta_cuisine_dict = {}
for city in model_data.city.unique():
    rev_delta_cuisine_dict.update({city: {'cuisine_style_count': model_data.loc[(model_data['city']==city)].groupby('cuisine_style_count')['reviews_date_delta'].median().reset_index().to_dict('index')}})
        
rev_delta_cuisine_dict = {k: v['cuisine_style_count'] for k, v in rev_delta_cuisine_dict.items()}
rev_delta_cuisine_dict = {key: {v['cuisine_style_count']: v['reviews_date_delta'] for k, v in rev_delta_cuisine_dict[key].items()} for key, value in rev_delta_cuisine_dict.items()}


rev_delta_max_cuisine_dict = {}
for city in model_data.city.unique():
    rev_delta_max_cuisine_dict.update({city: {'cuisine_style_count': model_data.loc[(model_data['city']==city)].groupby('cuisine_style_count')['reviews_date_delta_max'].median().reset_index().to_dict('index')}})
        
rev_delta_max_cuisine_dict = {k: v['cuisine_style_count'] for k, v in rev_delta_max_cuisine_dict.items()}
rev_delta_max_cuisine_dict = {key: {v['cuisine_style_count']: v['reviews_date_delta_max'] for k, v in rev_delta_max_cuisine_dict[key].items()} for key, value in rev_delta_max_cuisine_dict.items()}

    
df_test['reviews_date_delta_old'] = df_test['reviews_date_delta']
df_test['reviews_date_delta_max_old'] = df_test['reviews_date_delta_max']

df_test['reviews_date_delta'] = df_test.apply(lambda x: rev_delta_cuisine_dict[x['city']][x['cuisine_style_count']] if math.isnan(x['reviews_date_delta_old'])==True else x['reviews_date_delta'], axis=1)
df_test['reviews_date_delta_max'] = df_test.apply(lambda x: rev_delta_max_cuisine_dict[x['city']][x['cuisine_style_count']] if math.isnan(x['reviews_date_delta_max_old'])==True else x['reviews_date_delta_max'], axis=1)
    


NaNs processing started


In [19]:
# FEATURE ENGINEERING
    
print('Feature processing started')

# datasets' dividing
df_test['cuisine_style_count_old'] = df_test['cuisine_style'].dropna().apply(lambda x: len(x))
df_test['rest_type_count_old'] = df_test['rest_type_old'].dropna().apply(lambda x: len(x.split(', ')))
df_test['area_food_count_old'] = df_test['area_food_old'].dropna().apply(lambda x: len(x.split(', ')))
df_test['country_food_count_old'] = df_test['country_food_old'].dropna().apply(lambda x: len(x.split(', ')))
df_test['spcialities_count_old'] = df_test['spcialities_old'].dropna().apply(lambda x: len(x.split(', ')))

    
features_dict = {'common_features': ['rating', 'city', 'cuisine_style', 'ranking', 'city_count'],
                 'old_features': ['price_range_old', 'number_of_reviews_old', 'cuisine_style_count_old'],
                 'new_features': ['price_range', 'number_of_reviews', 'cuisine_style_count'],
                 'add_com_features': ['chain', 'chain_scale', 'name_city_count'],
                 'add_old_features': ['rest_type_old', 'area_food_old', 'country_food_old', 'spcialities_old',
                                      'rest_type_count_old', 'area_food_count_old', 'country_food_count_old', 'spcialities_count_old',
                                      'reviews_date_delta_old', 'reviews_date_delta_max_old'
                                     ],
                 'add_new_features': ['rest_type', 'area_food', 'country_food', 'spcialities',
                                      'rest_type_count', 'area_food_count', 'country_food_count', 'spcialities_count', 
                                      'reviews_date_delta', 'reviews_date_delta_max'
                                     ],
                 'not_relvant': ['restaurant_id', 'reviews', 'url_ta', 'id_ta',
                                 'name', 'region', 'g_code', 'gcode_count', 
                                 'reviews_date', 'reviews_date_last', 'reviews_text',
                                ]
                  }

print('Data columns count:', len(list(df_test.columns)))
df_test['ranking_city_scaled'] = df_test.apply(lambda x: round(x['ranking']/x['city_count'], 2), axis=1)

    
# normalization

# since the samples are balanced, we can normalize ranking as follows: 
df_test['ranking_norm'] = df_test.groupby('city')['ranking'].transform(lambda x: (x-x.min())/(x.max()-x.min()))
df_test['ranking_norm_log'] = df_test['ranking_norm'].apply(lambda x: np.log(x+1))

df_test['ranking_log'] = df_test['ranking'].apply(lambda x: np.log(x))
df_test['ranking_log_norm'] = df_test.groupby('city')['ranking_log'].transform(lambda x: (x-x.min())/(x.max()-x.min()))
df_test.drop('ranking_log', axis=1, inplace=True)

    
    
from sklearn.preprocessing import MinMaxScaler

for col in ['number_of_reviews_old', 'reviews_date_delta_old', 'reviews_date_delta_max_old']:
    scaler = MinMaxScaler()
    col_train = scaler.fit_transform(np.array(model_data[col]).reshape(-1, 1))
    col_test = scaler.transform(np.array(df_test[col]).reshape(-1, 1))
    
    indexes_test = pd.RangeIndex(start=0, stop=len(col_test))
    
    new_col_df = pd.DataFrame(data=col_test, index=indexes_test, columns=[col+'_norm'])
    
    df_test = pd.concat([df_test, new_col_df], axis=1)

    
features_dict.update({'norm_features': list(df_test.columns)[45:52]})
print('Data columns count:', len(list(df_test.columns)))



# statistics
    
print('Statistic features creating started')
    
# For some numeric features, we will calculate statistics on the bundle city & categorical feature
combinations = list(product(['ranking'], ['price_range', 'chain_scale'], ['mean'])) # cuisine_style
for comb in combinations:
    #              get_stat_dict(data, cat, num, stat)
    cat_num_dict = get_stat_dict(model_data, comb[1], comb[0], comb[2])
    df_test['_'.join(['city', str(comb[0]), str(comb[1]), str(comb[2])])] = df_test.apply(lambda x: cat_num_dict[x['city']][x[str(comb[1])]] if x['city'] in list(cat_num_dict.keys()) and x[str(comb[1])] in list(cat_num_dict[x['city']].keys()) and math.isnan(x[str(comb[1])])==False and math.isnan(x[str(comb[0])])==False else np.nan, axis=1)

for comb in combinations:
    #              get_stat_dict(data, cat, num, stat)
    cat_num_dict = get_stat_dict(df_test, comb[1], comb[0], comb[2])
    df_test['_'.join(['city', str(comb[0]), str(comb[1]), str(comb[2])])] = df_test.apply(lambda x: cat_num_dict[x['city']][x[str(comb[1])]] if math.isnan(x['_'.join(['city', str(comb[0]), str(comb[1]), str(comb[2])])])==True and x['city'] in list(cat_num_dict.keys()) and x[str(comb[1])] in list(cat_num_dict[x['city']].keys()) and math.isnan(x[str(comb[1])])==False and math.isnan(x[str(comb[0])])==False else x['_'.join(['city', str(comb[0]), str(comb[1]), str(comb[2])])], axis=1)


combinations = list(product(['number_of_reviews', 'reviews_date_delta', 'reviews_date_delta_max'], ['price_range', 'chain_scale'], ['median'])) # cuisine_style
for comb in combinations:
    #              get_stat_dict(data, cat, num, stat)
    cat_num_dict = get_stat_dict(model_data, comb[1], comb[0], comb[2])
    df_test['_'.join(['city', str(comb[0]), str(comb[1]), str(comb[2])])] = df_test.apply(lambda x: cat_num_dict[x['city']][x[str(comb[1])]] if x['city'] in list(cat_num_dict.keys()) and x[str(comb[1])] in list(cat_num_dict[x['city']].keys()) and math.isnan(x[str(comb[1])])==False and math.isnan(x[str(comb[0])])==False else np.nan, axis=1)

for comb in combinations:
    #              get_stat_dict(data, cat, num, stat)
    cat_num_dict = get_stat_dict(df_test, comb[1], comb[0], comb[2])
    df_test['_'.join(['city', str(comb[0]), str(comb[1]), str(comb[2])])] = df_test.apply(lambda x: cat_num_dict[x['city']][x[str(comb[1])]] if math.isnan(x['_'.join(['city', str(comb[0]), str(comb[1]), str(comb[2])])])==True and x['city'] in list(cat_num_dict.keys()) and x[str(comb[1])] in list(cat_num_dict[x['city']].keys()) and math.isnan(x[str(comb[1])])==False and math.isnan(x[str(comb[0])])==False else x['_'.join(['city', str(comb[0]), str(comb[1]), str(comb[2])])], axis=1)

    
combinations = list(product(['rating'], ['price_range', 'chain_scale', 'cuisine_style_count'], ['mean'])) # cuisine_style
for comb in combinations:
    #              get_stat_dict(data, cat, num, stat)
    cat_num_dict = get_stat_dict(model_data, comb[1], comb[0], comb[2])
    df_test['_'.join(['city', str(comb[0]), str(comb[1]), str(comb[2])])] = df_test.apply(lambda x: cat_num_dict[x['city']][x[str(comb[1])]] if x['city'] in list(cat_num_dict.keys()) and x[str(comb[1])] in list(cat_num_dict[x['city']].keys()) and math.isnan(x[str(comb[1])])==False else np.nan, axis=1)

    
features_dict.update({'stat_features': list(df_test.columns)[52:63]})
print('Data columns count:', len(list(df_test.columns)))
    
    
    
# additional features

# city can be represented, including the number of chains, as well as 'cheap', 'medium', 'expensive' restaurants
city_chain_dict = pd.DataFrame({'city': sorted(model_data.city.unique()),
                                'chain': model_data.loc[(model_data['chain']==1)].groupby('city').chain.value_counts().values
                                }).set_index('city').to_dict('index')

df_test['city_chain'] = df_test['city'].dropna().apply(lambda x: city_chain_dict[x]['chain'])

# 'cheap', 'medium', 'expensive'
city_price_range_dict = pd.DataFrame({'city': sorted(model_data.city.unique()),
                                      'cheap': model_data.loc[(model_data['price_range']==1.0)].groupby('city').price_range.value_counts().values,
                                      'medium': model_data.loc[(model_data['price_range']==2.5)].groupby('city').price_range.value_counts().values,
                                      'expensive': model_data.loc[(model_data['price_range']==4.0)].groupby('city').price_range.value_counts().values,
                                      }).set_index('city').to_dict('index')

df_test['city_cheap'] = df_test['city'].dropna().apply(lambda x: city_price_range_dict[x]['cheap'])
df_test['city_medium'] = df_test['city'].dropna().apply(lambda x: city_price_range_dict[x]['medium'])
df_test['city_expensive'] = df_test['city'].dropna().apply(lambda x: city_price_range_dict[x]['expensive'])


# Let's reduce the parametrs correlated with name_count
df_test['city_chain_scaled'] = df_test.apply(lambda x: x['city_chain']/x['city_count'], axis=1)
df_test['city_cheap_scaled'] = df_test.apply(lambda x: x['city_cheap']/x['city_count'], axis=1)
df_test['city_medium_scaled'] = df_test.apply(lambda x: x['city_medium']/x['city_count'], axis=1)
df_test['city_expensive_scaled'] = df_test.apply(lambda x: x['city_expensive']/x['city_count'], axis=1)
df_test['rev_num_city_scaled_old'] = df_test.apply(lambda x: round(x['number_of_reviews_old']/x['city_count'], 2) if math.isnan(x['number_of_reviews_old'])==False else np.nan, axis=1)

    
# reviews_text
pos_list = list(set(lists_dict['pos_list']))
neg_list = list(set(lists_dict['neg_list']))

# count the number of positive and negative words in the reviews
df_test['reviews_words_list'] = df_test['reviews_text'].dropna().apply(get_words_list)
df_test['reviews_pos_score'] = df_test['reviews_words_list'].dropna().apply(get_positive_score)
df_test['reviews_neg_score'] = df_test['reviews_words_list'].dropna().apply(get_negative_score)
df_test = df_test.drop('reviews_words_list', axis=1)

features_dict.update({'count_features': list(df_test.columns)[63:74]})
print('Data columns count:', len(list(df_test.columns)))
    
    
# Polynomial Features
    
print('Polynomial features creating started')

from sklearn.preprocessing import PolynomialFeatures

P = df_test[['ranking', 'city_count', 'number_of_reviews_old']].copy().fillna(0)
poly = PolynomialFeatures(2) # , interaction_only=True
P_poly = poly.fit_transform(P)

P_poly_df = pd.DataFrame(P_poly,
                         columns = ['1', 'ranking', 'city_count', 'number_of_reviews_old', 
                                    'ranking^2', 'ranking*city_count', 'ranking*number_of_reviews_old',
                                    'city_count^2', 'city_count*number_of_reviews_old', 'number_of_reviews_old^2'
                                   ]
                         )
poly_features = list(P_poly_df.columns)[4:] # 6

df_test = pd.concat([df_test, P_poly_df[poly_features]], axis=1)
df_test['ranking_sqrt'] = df_test['ranking'].dropna().apply(lambda x: np.sqrt(x))

features_dict.update({'nlarge_poly_features': list(df_test.columns)[74:81]})


P = df_test[['ranking_city_scaled', 'number_of_reviews_old', 'reviews_date_delta_max_old']].copy().fillna(0)
poly = PolynomialFeatures(2) # , interaction_only=True
P_poly = poly.fit_transform(P)

P_poly_df = pd.DataFrame(P_poly,
                         columns = ['1', 'ranking_city_scaled', 'number_of_reviews_old', 'reviews_date_delta_max_old', 
                                    'ranking_city_scaled^2', 'ranking_city_scaled*number_of_reviews_old', 'ranking_city_scaled*reviews_date_delta_max_old',
                                    'number_of_reviews_old^2', 'number_of_reviews_old*reviews_date_delta_max_old', 'reviews_date_delta_max_old^2'
                                   ]
                         )
poly_features = list(P_poly_df.columns)[4:]

df_test = pd.concat([df_test, P_poly_df[poly_features]], axis=1)

features_dict.update({'other_poly_features': list(df_test.columns)[81:87]})
print('Data columns count:', len(list(df_test.columns)))

    
    
# Dummy
    
    
print('Dummy features creating started')

# Cuisine Style

# city - cuisine_style
city_style = model_data.groupby('city')['cuisine_style'].apply(list).reset_index()
city_style['cuisine_style'] = city_style['cuisine_style'].apply(cuisine_count_dict)
city_style_dict_1 = city_style.set_index('city').to_dict('index')
city_style_dict_1 = {k: v['cuisine_style'] for k, v in city_style_dict_1.items()}
# 'Amsterdam': {'European': 501, 'Vegetarian Friendly': 469, 'Dutch': 286, ... }

city_style = df_test.groupby('city')['cuisine_style'].apply(list).reset_index()
city_style['cuisine_style'] = city_style['cuisine_style'].apply(cuisine_count_dict)
city_style_dict_0 = city_style.set_index('city').to_dict('index')
city_style_dict_0 = {k: v['cuisine_style'] for k, v in city_style_dict_0.items()}


style_list = []
for cuisine in model_data['cuisine_style'].copy().dropna():
    for style in cuisine:
        if not(style in style_list):
            style_list.append(style)
        
df_test['cuisine_style'] = df_test['cuisine_style'].fillna('-')

# dummy features with counts
for item in style_list:
    df_test['_'.join([str(item), 'count'])] = df_test.apply(lambda x: city_style_dict_1[x['city']][item] if item in x['cuisine_style'] and item in city_style_dict_1[x['city']] else (city_style_dict_0[x['city']][item] if item in x['cuisine_style'] and item in city_style_dict_0[x['city']] else 0), axis=1)

features_dict.update({'cuisine_count_dummy': list(df_test.columns)[87:]})
print('Data columns count:', len(list(df_test.columns)))
    

# city

n_col = len(list(df_test.columns))
df_test = pd.get_dummies(df_test, columns=['city'], dummy_na=False)

features_dict.update({'city_dummy': list(df_test.columns)[n_col-1:]})
print('Data columns count:', len(list(df_test.columns)))
features_dict.update({'common_features': remove_elments(features_dict['common_features'], ['city'])})
    
    
    
# data for model


# all additional with statistics & 3 norm & 3 large + 3 other polynomial without original & cuisine count + city dummy with NaNs
data_num = df_test[remove_elments(get_data_columns([get_data_columns_dict(['common_features', 'old_features', 'add_com_features', 'add_old_features', 'count_features', 'stat_features', 'nlarge_poly_features', 'other_poly_features', 'cuisine_count_dummy', 'city_dummy']), ['number_of_reviews_old_norm', 'reviews_date_delta_old_norm', 'reviews_date_delta_max_old_norm']]), ['ranking_sqrt', 'ranking', 'city_count', 'number_of_reviews_old', 'reviews_date_delta_max_old'])].select_dtypes(include='number').copy()
data_num = data_num.fillna(0)                                               

data_num.to_csv('data_for_model.csv', index=False)
    
    
    #return data_num


Feature processing started
Data columns count: 45
Data columns count: 52
Statistic features creating started
Data columns count: 63
Data columns count: 74
Polynomial features creating started
Data columns count: 87
Dummy features creating started
Data columns count: 143
Data columns count: 173


In [20]:
data_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 137 entries, rating to reviews_date_delta_max_old_norm
dtypes: float64(42), int64(64), uint8(31)
memory usage: 8.4 MB


In [21]:
data_num.sample(5)

,rating,price_range_old,cuisine_style_count_old,chain,chain_scale,name_city_count,rest_type_count_old,area_food_count_old,country_food_count_old,spcialities_count_old,...,city_Paris,city_Prague,city_Rome,city_Stockholm,city_Vienna,city_Warsaw,city_Zurich,number_of_reviews_old_norm,reviews_date_delta_old_norm,reviews_date_delta_max_old_norm
8775,0,0.0,0.0,0,1,1,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0.000518,0.000000,0.000000
6235,0,2.5,1.0,0,1,1,1.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0.005591,0.004054,0.029452
8134,0,2.5,5.0,0,1,1,2.0,1.0,1.0,1.0,...,1,0,0,0,0,0,0,0.009836,0.007172,0.027080
5775,0,2.5,2.0,0,1,1,0.0,1.0,1.0,0.0,...,0,0,0,0,0,1,0,0.001864,0.055815,0.041510
4937,0,2.5,2.0,0,1,1,0.0,1.0,1.0,0.0,...,0,1,0,0,0,0,0,0.001035,0.016526,0.117612


# Model

In [ ]:
import pickle

with open('/kaggle/input/bella-k-sf-tripadvisor-rating-best-model/best_model.pkl', 'rb') as pkl_file:
    best_model = pickle.load(pkl_file)

best_model

In [ ]:
data_num = data_num.drop('rating', axis=1)
predict_submission = best_model.predict(data_num)
predict_submission

In [ ]:
# rounding predict with pitch 0.5
predict_submission_round = []
for item in predict_submission:
    predict_submission_round.append(round(item/0.5)*0.5)

predict_submission_array = np.asarray(predict_submission_round)
predict_submission_array

# Submision

In [ ]:
sample_submission.sample(5)

In [ ]:
sample_submission['Rating'] = predict_submission_array
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()